# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.impute import SimpleImputer
clf = SimpleImputer()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


SimpleImputer()

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {

		tTable lTable;

		lTable["Feature_0"] = { ((Feature_0 == std::any()) ?  ( 5.843333333333334 ) : ( Feature_0)) };
		lTable["Feature_1"] = { ((Feature_1 == std::any()) ?  ( 3.0573333333333337 ) : ( Feature_1)) };
		lTable["Feature_2"] = { ((Feature_2 == std::any()) ?  ( 3.7580000000000005 ) : ( Feature_2)) };
		lTable["Feature_3"] = { ((Feature_3 == std::any()) ?  ( 1.1993333333333336 ) : ( Feature_3)) };

		return lTable;
	}

	tTable compute_model_outputs_from_table( tTable const & iTable) {
		tTable lTable = compute_features(iTable.at("Featu

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140680683702928.exe /tmp/sklearn2sql_cpp_140680683702928.cpp'
idx,Feature_0,Feature_1,Feature_2,Feature_3
0,5.10000000000000,3.50000000000000,1.40000000000000,0.20000000000000
1,4.90000000000000,3.00000000000000,1.40000000000000,0.20000000000000
2,4.70000000000000,3.20000000000000,1.30000000000000,0.20000000000000
3,4.60000000000000,3.10000000000000,1.50000000000000,0.20000000000000
4,5.00000000000000,3.60000000000000,1.40000000000000,0.20000000000000
5,5.40000000000000,3.90000000000000,1.70000000000000,0.40000000000000
6,4.60000000000000,3.40000000000000,1.40000000000000,0.30000000000000
7,5.00000000000000,3.40000000000000,1.50000000000000,0.20000000000000
8,4.40000000000000,2.90000000000000,1.40000000000000,0.20000000000000
9,4.90000000000000,3.10000000000000,1.50000000000000,0.10000000000000
10,5.40000000000000,3.70000000000000,1.50000000000000,0.20000000000000
11,4.80000000000000,3.40000000000000,1.6000

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,5.8,2.8,5.1,2.4
74,74,6.4,2.9,4.3,1.3
9,9,4.9,3.1,1.5,0.1
88,88,5.6,3.0,4.1,1.3
25,25,5.0,3.0,1.6,0.2
5,5,5.4,3.9,1.7,0.4
48,48,5.3,3.7,1.5,0.2
117,117,7.7,3.8,6.7,2.2
83,83,6.0,2.7,5.1,1.6
105,105,7.6,3.0,6.6,2.1


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,5.8,2.8,5.1,2.4
74,74,6.4,2.9,4.3,1.3
9,9,4.9,3.1,1.5,0.1
88,88,5.6,3.0,4.1,1.3
25,25,5.0,3.0,1.6,0.2
5,5,5.4,3.9,1.7,0.4
48,48,5.3,3.7,1.5,0.2
117,117,7.7,3.8,6.7,2.2
83,83,6.0,2.7,5.1,1.6
105,105,7.6,3.0,6.6,2.1


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Feature_0_skl,Feature_1_skl,Feature_2_skl,Feature_3_skl,idx_cpp,Feature_0_cpp,Feature_1_cpp,Feature_2_cpp,Feature_3_cpp
114,114,5.8,2.8,5.1,2.4,114,5.8,2.8,5.1,2.4
74,74,6.4,2.9,4.3,1.3,74,6.4,2.9,4.3,1.3
9,9,4.9,3.1,1.5,0.1,9,4.9,3.1,1.5,0.1
88,88,5.6,3.0,4.1,1.3,88,5.6,3.0,4.1,1.3
25,25,5.0,3.0,1.6,0.2,25,5.0,3.0,1.6,0.2
5,5,5.4,3.9,1.7,0.4,5,5.4,3.9,1.7,0.4
48,48,5.3,3.7,1.5,0.2,48,5.3,3.7,1.5,0.2
117,117,7.7,3.8,6.7,2.2,117,7.7,3.8,6.7,2.2
83,83,6.0,2.7,5.1,1.6,83,6.0,2.7,5.1,1.6
105,105,7.6,3.0,6.6,2.1,105,7.6,3.0,6.6,2.1


In [14]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())

count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
